In [22]:
%pip install newspaper requests pandas beautifulsoup4

  Using cached newspaper-0.1.0.7.tar.gz (176 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [1 lines of output]
      WARNING! You are attempting to install newspaper's python2 repository on python3. PLEASE RUN `$ pip3 install newspaper3k` for python3 or `$ pip install newspaper` for python2
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Note: you may need to restart the kernel to use updated packages.


In [23]:
import requests
import pandas as pd
from datetime import datetime
import time
from bs4 import BeautifulSoup

In [24]:
COUNTRIES = [
    'United States', 'Germany', 'United Kingdom', 'France', 'Japan',
    'Canada', 'Australia', 'Brazil', 'India', 'South Korea', 'China',
    'Hong Kong', 'Taiwan', 'Netherlands', 'Switzerland', 'Italy',
    'Spain', 'Sweden', 'Belgium', 'Norway', 'Denmark', 'Finland',
    'Portugal', 'Greece', 'Poland', 'Turkey','South Africa', 'Nigeria',
    'Egypt', 'Kenya', 'Russia', 'Myanmar'
]

In [25]:
def fetch_google_news(country, start_date, end_date):
    base_url = "https://news.google.com/rss/search"
    query = f"q={country}+after:{start_date}+before:{end_date}"
    url = f"{base_url}?{query}&hl=en-US&gl=US&ceid=US:en"
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        return response.text
    except Exception as e:
        print(f"Error fetching Google News for {country}: {e}")
        return None


In [ ]:
def extract_article_text(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        for tag in soup(["script", "style", "noscript"]):
            tag.decompose()

        article = soup.find('article')
        if not article:
            possible_blocks = soup.find_all(['div', 'section'], class_=lambda c: c and 'content' in c.lower())
            article = max(possible_blocks, key=lambda tag: len(tag.get_text(strip=True)), default=soup.body)

        text = article.get_text(separator='\n', strip=True)
        return text
    except:
        return 'Content not available'


In [27]:
def parse_news_feed(feed_xml, country):
    articles = []
    try:
        soup = BeautifulSoup(feed_xml, 'xml')
        for item in soup.find_all('item')[:10]:  # Limit to 10 articles per country
            try:
                article_url = item.link.text.split('&url=')[-1] if '&url=' in item.link.text else item.link.text
                articles.append({
                    'country': country,
                    'date': item.pubDate.text[:10] if item.pubDate else '',
                    'source': item.source.text if item.source else '',
                    'title': item.title.text,
                    'description': extract_article_text(article_url),
                    'url': article_url
                })
            except Exception as e:
                print(f"Error processing article for {country}: {e}")
                continue
    except Exception as e:
        print(f"Error parsing feed for {country}: {e}")
    return articles


In [ ]:

start_date = datetime(2025, 4, 1).strftime('%Y-%m-%d')
end_date = datetime(2025, 4, 7).strftime('%Y-%m-%d')

all_articles = []

for country in COUNTRIES:
    print(f"Fetching news for {country}...")
    feed = fetch_google_news(country, start_date, end_date)
    if feed:
        articles = parse_news_feed(feed, country)
        all_articles.extend(articles)
    time.sleep(3)

df = pd.DataFrame(all_articles)
df.to_csv('global_finance_news.csv', index=False)
print(f"Saved {len(all_articles)} articles to april_2025_news.csv")

Fetching news for United States...
Fetching news for Germany...
